In [ ]:
# Source the package setup script
source("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/scripts/00_setup_packages.R");

# Data Cleaning for Models 1, 2, 3 and Substrate Experiment Model

## Overview

This pipeline focuses on preparing a dataset for Models 1, 1_exp, 2, and 3 (i.e., occurrence models). The main goal is to clean and format the data, ensuring it meets the requirements of the analyses. These steps make the dataset consistent, interpretable, and suitable for statistical modeling.


### 1. Load data

We start by loading the raw data from the BMC_BCPD_2021_FieldSurvey.xlsx and SubstrateExperimentData.xlsx file. These datasets contain information about the occurrence of Podocerus cristatus in the field and ex situ, respectively.

In [ ]:
data_m1m2m3 <- read_excel("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/raw/BMC_BCPD_2021_FieldSurvey.xlsx", sheet = "Data_m1m2m3")

data_exp <- data.frame(read_excel("C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/raw/SubstrateExperimentData.xlsx", sheet = "Data_R"))

---

### 2. Clean data

The dataset is cleaned in several steps:

1. **Remove Poor-Quality Data**: Exclude bad photos and irrelevant rows.
2. **Select Variables**: Keep only the columns relevant to Models.
3. **Filter Rows**: Subset data by lifestage (Model 2, 3, & 4) and Pod presence (Models 3 & 4).
4. **Rename Columns**: Simplify and clarify variable names.
5. **Format Variables**: Convert continuous variables to numeric and categorical ones to factors.
6. **Rename Categories**: Simplify and clarify microhabitat category names.
7. **Remove Missing Values**: Ensure no NAs remain in critical columns.

<div style='border-left: 5px solid #0078D7; padding: 10px; background-color: #F0F8FF;'>
  <strong>💡 Note:</strong> To enhance statistical power in Models 3 and 4, we have consolidated the non-significant Microhabitat categories identified in Model 2 (see "02_model_2.ipynb"). This simplified variable was called "Microhabitat_cat" in the Excel sheet and used as the Microhabitat variable in the data below for Model 3 and 4.
</div>
<br>

In [ ]:
# Step 1: Select variables of interest
data_m1_subset <- data_m1m2m3 %>%
    dplyr::select(Site, Quadrat, Microhabitat, Nudie_aeolid_presence, Pod_presence, Juvenile_presence, Adult_presence, Pod_presence)

data_exp_subset <- data_exp %>%
    dplyr::select(Chamber, TOD, Hour, Substrate, Sex, Count)

data_m2_subset <- data_m1m2m3 %>%
    dplyr::select(Site, Quadrat, Microhabitat_cat, Nudie_aeolid_presence, Pod_presence, Male_presence, Female_presence, Juvenile_presence, Adult_presence, Pod_presence)

data_m3_subset <- data_m1m2m3 %>%
    dplyr::select(Site, Quadrat, Microhabitat_cat, Nudie_aeolid_presence, Pod_presence, RB_presence, W_presence, Bright_presence, Juvenile_presence, Adult_presence, Pod_presence)


# Step 2: Filter rows based on Lifestage (Model 2,3, & 4) and Pod presence (Model 3 & 4)
data_m1_subset <- data_m1_subset %>%
    filter(Juvenile_presence != "1")

data_m2_subset <- data_m2_subset %>%
    filter(Juvenile_presence != "1") %>%
    filter(Pod_presence == "1")

data_m3_subset <- data_m3_subset %>%
    filter(Juvenile_presence != "1") %>%
    filter(Pod_presence == "1")



# Step 3: Rename variables for clarity
data_m1_subset_renamed <- data_m1_subset %>%
    rename(
        Nudibranch = Nudie_aeolid_presence,
        Pod = Pod_presence
    )

data_exp_subset_renamed <- data_exp_subset %>%
    rename(
        Time = Hour,
        Microhabitat = Substrate
    )

data_m2_subset_renamed <- data_m2_subset %>%
    rename(
        Microhabitat = Microhabitat_cat,
        Nudibranch = Nudie_aeolid_presence,
        Male = Male_presence,
        Female = Female_presence
    )

data_m3_subset_renamed <- data_m3_subset %>%
    rename(
        Microhabitat = Microhabitat_cat,
        Nudibranch = Nudie_aeolid_presence,
        Red_Beige = RB_presence,
        White = W_presence,
        Bright = Bright_presence
    )


# Step 4: Convert continuous variables to numeric, if not already done so
columns_to_convert_exp <- c("Count")
columns_to_convert_exp <- c("Time")

# Convert columns
data_exp_subset_renamed <- data_exp_subset_renamed %>%
    mutate(across(all_of(columns_to_convert_exp), as.numeric))

# Step 5: Convert binary/categorical variables to factors, if not already done so
# Define the columns to convert to factor
columns_to_convert_m1 <- c("Site", "Quadrat", "Microhabitat", "Pod", "Nudibranch")
columns_to_convert_exp <- c("Chamber", "TOD", "Microhabitat", "Sex")
columns_to_convert_m2 <- c("Site", "Quadrat", "Microhabitat", "Nudibranch", "Male", "Female")
columns_to_convert_m3 <- c("Site", "Quadrat", "Microhabitat", "Nudibranch", "Red_Beige", "White", "Bright")

# Convert columns
data_m1_subset_renamed <- data_m1_subset_renamed %>%
    mutate(across(all_of(columns_to_convert_m1), as.factor))

data_exp_subset_renamed <- data_exp_subset_renamed %>%
    mutate(across(all_of(columns_to_convert_exp), as.factor))

data_m2_subset_renamed <- data_m2_subset_renamed %>%
    mutate(across(all_of(columns_to_convert_m2), as.factor))

data_m3_subset_renamed <- data_m3_subset_renamed %>%
    mutate(across(all_of(columns_to_convert_m3), as.factor))



# Step 6: Rename Microhabitat categories for clarity
data_m1_subset_renamed <- data_m1_subset_renamed %>%
    mutate(Microhabitat = fct_recode(Microhabitat,
        Sertulariidae_Bryozoa = "Sertulariidae_BRYO",
        Red_Algae = "Algae",
        Other_Hydroids = "Other"
    ))

data_exp_subset_renamed <- data_exp_subset_renamed %>%
    mutate(Microhabitat = fct_recode(Microhabitat,
        Red_Algae = "RedAlgae",
        Sertulariidae = "Sertulariid"
    ))

data_m2_subset_renamed <- data_m2_subset_renamed %>%
    mutate(Microhabitat = fct_recode(Microhabitat,
        Hydroid_Bryozoa = "Hydroid_bryozoan",
        Red_Algae = "Algae"
    ))

data_m3_subset_renamed <- data_m3_subset_renamed %>%
    mutate(Microhabitat = fct_recode(Microhabitat,
        Hydroid_Bryozoa = "Hydroid_bryozoan",
        Red_Algae = "Algae"
    ))



# Step 7: delete NA rows
data_m1_clean <- data_m1_subset_renamed %>%
    filter(!is.na(Pod))

data_exp_clean <- data_exp_subset_renamed %>%
    filter(!is.na(Time))

data_m2_clean <- data_m2_subset_renamed %>%
    filter(!is.na(Male) & !is.na(Female))

data_m3_clean <- data_m3_subset_renamed %>%
    filter(!is.na(Red_Beige) & !is.na(White) & !is.na(Bright))

# NOTE: "NAs introduced by coercion" isn't an error. It's just a warning to let you know that some strings don't represent numbers. Those are correctly changed to NA.

In [ ]:
table_1 <- data_m1_clean %>%
  mutate(across(where(is.numeric), ~ signif(.x, digits = 3))) %>%
  kbl(caption = "Raw Data for Model 2") %>%
  kable_classic(full_width = FALSE, html_font = "Cambria") %>%
  as.character()  # Convert to raw HTML

# Add a scrollable div
table_1 <- paste0("
<style>
  .scrollable-table {
    max-height: 300px; /* Adjust height as needed */
    overflow-y: auto;
    border: 1px solid #ddd;
    padding: 5px;
  }
</style>

<div class='scrollable-table'>", table_1, "</div>
")


table_2 <- data_exp_clean %>%
  mutate(across(where(is.numeric), ~ signif(.x, digits = 3))) %>%
  kbl(caption = "Raw Data for Substrate Experiment Model") %>%
  kable_classic(full_width = FALSE, html_font = "Cambria") %>%
  as.character()  # Convert to raw HTML

# Add a scrollable div
table_2 <- paste0("
<style>
  .scrollable-table {
    max-height: 300px; /* Adjust height as needed */
    overflow-y: auto;
    border: 1px solid #ddd;
    padding: 5px;
  }
</style>

<div class='scrollable-table'>", table_2, "</div>
")


table_3 <- data_m2_clean %>%
  mutate(across(where(is.numeric), ~ signif(.x, digits = 3))) %>%
  kbl(caption = "Raw Data for Model 3") %>%
  kable_classic(full_width = FALSE, html_font = "Cambria") %>%
  as.character()  # Convert to raw HTML

# Add a scrollable div
table_3 <- paste0("
<style>
  .scrollable-table {
    max-height: 300px; /* Adjust height as needed */
    overflow-y: auto;
    border: 1px solid #ddd;
    padding: 5px;
  }
</style>

<div class='scrollable-table'>", table_3, "</div>
")


table_4 <- data_m3_clean %>%
  mutate(across(where(is.numeric), ~ signif(.x, digits = 3))) %>%
  kbl(caption = "Raw Data for Model 4") %>%
  kable_classic(full_width = FALSE, html_font = "Cambria") %>%
  as.character()  # Convert to raw HTML

# Add a scrollable div
table_4 <- paste0("
<style>
  .scrollable-table {
    max-height: 300px; /* Adjust height as needed */
    overflow-y: auto;
    border: 1px solid #ddd;
    padding: 5px;
  }
</style>

<div class='scrollable-table'>", table_4, "</div>
")


In [ ]:

my_tabs <- '
<style>
/* Basic container styling */
.tabs-container {
  width: 100%;
  margin: 1em 0;
}

/* Hide the radio inputs (we only show their labels as tabs) */
.tabs-container input[type="radio"] {
  display: none;
}

/* The “tab-label” styling: looks like a tab */
.tab-label {
  display: inline-block;
  padding: 10px;
  margin-right: 2px;
  background: #eee;
  border: 1px solid #ccc;
  cursor: pointer;
  border-bottom: none;
}

/* The active tab label */
.tab-label-active {
  background: #fff;
}

/* The panel that holds table content */
.tab-content {
  border: 1px solid #ccc;
  padding: 10px;
  display: none;
}

/* For each radio input, show its corresponding content when checked */
#tab1:checked ~ #content1,
#tab2:checked ~ #content2,
#tab3:checked ~ #content3,
#tab4:checked ~ #content4 {
  display: block;
}

/* Also style the label of the checked radio as “active” using the :checked + label technique */
#tab1:checked + label[for="tab1"],
#tab2:checked + label[for="tab2"],
#tab3:checked + label[for="tab3"],
#tab4:checked + label[for="tab4"] {
  background: #fff;
  border-bottom: none;
}
</style>

<div class="tabs-container">

  <!-- 1) Tab radio + label -->
  <input type="radio" name="tabs" id="tab1" checked>
  <label class="tab-label" for="tab1">Table 1</label>

  <!-- 2) Tab radio + label -->
  <input type="radio" name="tabs" id="tab2">
  <label class="tab-label" for="tab2">Table 2</label>

  <!-- 3) Tab radio + label -->
  <input type="radio" name="tabs" id="tab3">
  <label class="tab-label" for="tab3">Table 3</label>

  <!-- 4) Tab radio + label -->
  <input type="radio" name="tabs" id="tab4">
  <label class="tab-label" for="tab4">Table 4</label>


  <!-- Content for each tab -->
  <div class="tab-content" id="content1">REPLACE_WITH_table_1</div>
  <div class="tab-content" id="content2">REPLACE_WITH_table_2</div>
  <div class="tab-content" id="content3">REPLACE_WITH_table_3</div>
  <div class="tab-content" id="content4">REPLACE_WITH_table_4</div>
</div>
'

# Now do the replacements for each table
my_tabs <- gsub("REPLACE_WITH_table_1", table_1, my_tabs)
my_tabs <- gsub("REPLACE_WITH_table_2", table_2, my_tabs)
my_tabs <- gsub("REPLACE_WITH_table_3", table_3, my_tabs)
my_tabs <- gsub("REPLACE_WITH_table_4", table_4, my_tabs)

IRdisplay::display_html(my_tabs)

---

### 3. Export cleaned data

The cleaned datasets are saved as .csv files for downstream analyses.

In [ ]:
write.csv(data_m1_clean, "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/cleaned/data_m1_clean.csv", row.names = FALSE)


write.csv(data_exp_clean, "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/cleaned/data_exp_clean.csv", row.names = FALSE)


write.csv(data_m2_clean, "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/cleaned/data_m2_clean.csv", row.names = FALSE)


write.csv(data_m3_clean, "C:/Users/bmc82/Documents/UF/PhD_Projects/DISSERTATION_MANUSCRIPT/Chapter_3/chapter3_data_analysis/data/cleaned/data_m3_clean.csv", row.names = FALSE)